In [6]:
import sys
sys.path.append('FFNSL')
sys.path.append('FFNSL/examples/follow_suit/')

In [7]:
import torch
import torch.nn as nn
import json
import sys
import math

from experiment_config import custom_args, process_custom_args
from follow_suit_dataset import load_follow_suit_data
from feature_extractor.dataset import load_data
from feature_extractor.network import PlayingCardNet
from ilp_config import background_knowledge, mode_declarations
from os.path import dirname, realpath

In [8]:
background_knowledge = '''
diff_color(red,blue).
diff_color(blue,red).
diff_color(red,yellow).
diff_color(yellow,red).
diff_color(blue,yellow).
diff_color(yellow,blue).
diff_shape(circle,square).
diff_shape(square,circle).
diff_shape(circle,triangle).
diff_shape(triangle,circle).
diff_shape(square,triangle).
diff_shape(triangle,square).

same_shape_pair(X,Y):-shape(X,Z),shape(Y,Z).
same_color_pair(X,Y):-color(X,Z),color(Y,Z).
diff_shape_pair(X,Y):-shape(X,Z),shape(Y,W),diff_shape(Z,W).
diff_color_pair(X,Y):-color(X,Z),color(Y,W),diff_color(Z,W).
in4(X):-in(O1,X),in(O2,X),in(O3,X),in(O4,X).
'''


mode_declarations = '''
#constant(image,img)
#constant(object,obj1)
#constant(object,obj2)
#constant(object,obj3)
#constant(object,obj4)
#constant(object,obj5)
#constant(object,obj6)

#constant(color,red)
#constant(color,blue)
#constant(color,yellow)
#constant(shape,circle)
#constant(shape,square)
#constant(shape,triangle)

#modeh(kp(var(image))).
#modeb(1, in4(var(image)).
#modeb(1, color(var(color))).
#modeb(1, shape(var(shape))).
#modeb(2, same_color_pair(var(object),var(object))).
#modeb(1, diff_color_pair(var(object),var(object))).
#modeb(2, same_shape_pair(var(object),var(object))).
#modeb(1, diff_shape_pair(var(object),var(object))).
#modeb(1, closeby(var(object),var(object))).
#modeb(1, online(var(object),var(object),var(object),var(object),var(object))).
'''

'''
    modeb_list = [
        ModeDeclaration('body', obj_num, lang.get_pred_by_name(
            'in'), [m_object, p_image]),
        ModeDeclaration('body', 1, lang.get_pred_by_name(
            'color'), [p_object, s_color]),
        ModeDeclaration('body', 1, lang.get_pred_by_name(
            'shape'), [p_object, s_shape]),
        ModeDeclaration('body', 2, lang.get_pred_by_name(
            'same_color_pair'), [p_object, p_object], ordered=False),
        ModeDeclaration('body', 2, lang.get_pred_by_name(
            'same_shape_pair'), [p_object, p_object], ordered=False),
        ModeDeclaration('body', 1, lang.get_pred_by_name(
            'diff_color_pair'), [p_object, p_object], ordered=False),
        ModeDeclaration('body', 1, lang.get_pred_by_name(
            'diff_shape_pair'), [p_object, p_object], ordered=False),
        ModeDeclaration('body', 1, lang.get_pred_by_name(
            'closeby'), [p_object, p_object], ordered=False),
        ModeDeclaration('body', 1, lang.get_pred_by_name('online'), [
                        p_object, p_object, p_object, p_object, p_object], ordered=False),
        # ModeDeclaration('body', 2, lang.get_pred_by_name('diff_shape_pair'), [p_object, p_object]),
    ]
'''

''

In [ ]:
def valuation_to_ILP_example_batch(V_0, atoms, labels, example_ids, th=0.5):
    examples = []
    for i, v in enumerate(V_0):
        label = labels[i]
        true_atoms = []
        for j in range(len(v)):
            if v[j] > th:
                true_atoms.append(atoms[j])
            examples.append((true_atoms,label,example_ids[i]))
        else:
            assert 0, "wrong label!"

    examples = atoms_to_ILASP_text(examples)
    return examples

def atoms_to_ILASP_text(atoms_list):
    for atoms, label, id in atoms_list:
        # positive example
        if label == 1:
            text = "#pos(\{{}@1 \},\{\}, \{\}\n".format(id) # penalty 1
            for atom in atoms:
                text += str(atom)
                text += ".\n"
        # negative example
        elif label == 0:
            text = "#neg(\{{}@1 \},\{\}, \{\}\n".format(id) # penalty 1
            for atom in atoms:
                text += str(atom)
                text += ".\n"
        text += "\})."


def predict():
    count_start = 0
    for data, target in data_loader:
        V_0 = ..
        atoms_to_ILASP_text(V_0)
    #TODO




In [ ]:

def create_ILP_example(data,
                       target,
                       preds,
                       example_id,
                       possible_classes,
                       net_type):

    classes = possible_classes.copy()
    # Build inclusion and exclusion sets
    inclusion = target.item()
    classes.remove(inclusion)
    inclusion_str = 'winner({0})'.format(inclusion)

    exclusion_str = ''
    for c in classes:
        exclusion_str += 'winner({0}), '.format(c)
    exclusion_str = exclusion_str[:-2]

    # Build context - get predictions for each card
    ctx = ''
    conf_scores = []
    for idx, card in enumerate(data):
        pred, conf = preds[str(card.item())+'.jpg']
        suit = pred[-1]
        rank = pred[:-1]
        ctx += '\tcard({0},{1},{2}).\n'.format(str(idx+1), rank, suit)
        conf_scores.append(conf)

    if 'constant' in net_type:
        penalty = 10
    else:
        penalty = calc_example_penalty(conf_scores)

    example = '#pos(eg(id{0})@{1}, {{ {2} }}, {{ {3} }}, {{\n'.format(example_id, penalty, inclusion_str, exclusion_str)
    example += ctx
    example += '}).'

    return example